# 94. CNNが苦手なケース：帰納バイアスの限界

## 学習目標

このノートブックでは、以下を学びます：

1. **CNNの帰納バイアスが合わないケース**
2. **回転・スケール変換への弱さ**
3. **長距離依存関係の問題**
4. **テクスチャバイアス**

## 目次

1. [帰納バイアスの限界](#section1)
2. [回転に対する弱さ](#section2)
3. [長距離依存関係](#section3)
4. [テクスチャバイアス](#section4)
5. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import rotate, zoom
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

<a id="section1"></a>
## 1. 帰納バイアスの限界

CNNの帰納バイアスは画像認識に強力ですが、すべての問題に適しているわけではありません。

### CNNが仮定していること

1. **局所的なパターンが重要** → 大域的な関係が重要な場合に弱い
2. **平行移動等変** → 回転・スケール変化には対応しない
3. **テクスチャが重要** → 形状ベースの認識に弱い可能性

In [ ]:
def visualize_cnn_limitations():
    """CNNの限界を可視化"""
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    
    # 1. 回転に対する非等変性
    ax = axes[0, 0]
    img = np.zeros((32, 32))
    img[10:22, 14:18] = 1  # 縦長の矩形
    
    ax.imshow(img, cmap='Blues')
    ax.set_title('回転に非等変\n同じ「縦棒」でも', fontsize=12)
    ax.axis('off')
    
    ax = axes[0, 1]
    img_rot = rotate(img, 45, reshape=False)
    ax.imshow(img_rot, cmap='Blues')
    ax.set_title('45°回転すると\n異なる特徴として検出', fontsize=12)
    ax.axis('off')
    
    ax = axes[0, 2]
    ax.text(0.5, 0.5, '回転等変でない\n\nデータ拡張で\n対処が一般的', 
           fontsize=12, ha='center', va='center', transform=ax.transAxes,
           bbox=dict(boxstyle='round', facecolor='lightyellow'))
    ax.axis('off')
    
    # 2. 長距離依存関係
    ax = axes[1, 0]
    img2 = np.zeros((32, 32))
    img2[4:8, 4:8] = 1
    img2[24:28, 24:28] = 1
    
    ax.imshow(img2, cmap='Reds')
    ax.annotate('', xy=(26, 26), xytext=(6, 6),
               arrowprops=dict(arrowstyle='<->', color='blue', lw=2))
    ax.set_title('長距離依存関係\nこの2つの関係は？', fontsize=12)
    ax.axis('off')
    
    ax = axes[1, 1]
    ax.text(0.5, 0.6, '受容野の限界\n\n初期層では\n遠い点を同時に\n見れない', 
           fontsize=11, ha='center', va='center', transform=ax.transAxes)
    ax.text(0.5, 0.2, '解決策:\n・深い層を使う\n・Attention機構', 
           fontsize=10, ha='center', va='center', transform=ax.transAxes,
           bbox=dict(boxstyle='round', facecolor='lightgreen'))
    ax.axis('off')
    
    ax = axes[1, 2]
    # テクスチャバイアスの例
    np.random.seed(42)
    texture = np.random.rand(32, 32) * 0.5
    # 猫の形（シルエット）にテクスチャを適用
    shape = np.zeros((32, 32))
    shape[8:24, 10:22] = 1  # 体
    shape[4:12, 12:20] = 1  # 頭
    
    combined = texture * shape
    ax.imshow(combined, cmap='gray')
    ax.set_title('テクスチャバイアス\nCNNは形状より\nテクスチャを重視しがち', fontsize=12)
    ax.axis('off')
    
    plt.suptitle('CNNの帰納バイアスの限界', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_cnn_limitations()

<a id="section2"></a>
## 2. 回転に対する弱さ

CNNは平行移動には等変ですが、回転には等変ではありません。

In [ ]:
def demonstrate_rotation_problem():
    """回転問題のデモ"""
    fig, axes = plt.subplots(2, 5, figsize=(18, 8))
    
    # 元画像（数字「6」風）
    img = np.zeros((28, 28))
    # 丸
    y, x = np.ogrid[:28, :28]
    center = (18, 14)
    r = 6
    mask = (x - center[0])**2 + (y - center[1])**2 < r**2
    img[mask] = 1
    # 上の棒
    img[4:18, 12:16] = 1
    
    angles = [0, 30, 60, 90, 180]
    
    for idx, angle in enumerate(angles):
        rotated = rotate(img, angle, reshape=False)
        
        axes[0, idx].imshow(rotated, cmap='gray_r')
        axes[0, idx].set_title(f'{angle}°回転', fontsize=12)
        axes[0, idx].axis('off')
        
        # 縦エッジフィルタの応答
        kernel = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=float)
        from scipy.signal import correlate2d
        response = correlate2d(rotated, kernel, mode='same')
        
        axes[1, idx].imshow(response, cmap='RdBu')
        axes[1, idx].set_title(f'縦エッジ応答', fontsize=10)
        axes[1, idx].axis('off')
    
    axes[0, 0].set_ylabel('入力', fontsize=12)
    axes[1, 0].set_ylabel('フィルタ応答', fontsize=12)
    
    plt.suptitle('回転に対するCNNの問題\n同じカーネルでは回転後のパターンを同様に検出できない', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("観察:")
    print("- 縦エッジカーネルは0°では縦エッジを検出")
    print("- 90°回転すると、元の縦エッジは横エッジになり検出されない")
    print("- 回転等変にするには回転したカーネルも必要")

demonstrate_rotation_problem()

In [ ]:
def show_rotation_solutions():
    """回転問題の解決策"""
    print("="*60)
    print("回転に対する解決策")
    print("="*60)
    
    solutions = [
        ("1. データ拡張", 
         "訓練時にランダム回転を適用",
         "最も一般的、簡単だが計算コスト増"),
        
        ("2. 回転等変ネットワーク",
         "Group Equivariant CNN (G-CNN)",
         "理論的に美しいが実装が複雑"),
        
        ("3. Spatial Transformer Network",
         "入力を正規化する層を学習",
         "柔軟だが学習が難しい場合も"),
        
        ("4. 回転不変な特徴",
         "Global Poolingで位置情報を捨てる",
         "情報損失があるが簡単")
    ]
    
    for name, desc, note in solutions:
        print(f"\n{name}")
        print(f"  説明: {desc}")
        print(f"  備考: {note}")

show_rotation_solutions()

<a id="section3"></a>
## 3. 長距離依存関係

CNNの受容野は層を重ねないと広がりません。初期層では遠く離れた点の関係を捉えられません。

In [ ]:
def demonstrate_long_range_problem():
    """長距離依存関係の問題"""
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # 例: 2つの点の関係
    ax = axes[0]
    img = np.zeros((32, 32))
    img[5:9, 5:9] = 1    # 左上
    img[23:27, 23:27] = 1  # 右下
    
    ax.imshow(img, cmap='Reds')
    ax.annotate('', xy=(25, 25), xytext=(7, 7),
               arrowprops=dict(arrowstyle='<->', color='blue', lw=2))
    ax.set_title('問題: 遠く離れた点の関係\n距離 ≈ 25ピクセル', fontsize=12)
    ax.axis('off')
    
    # 受容野の成長
    ax = axes[1]
    layers = range(1, 11)
    rf_3x3 = [1 + 2*n for n in layers]  # 3x3カーネル
    rf_5x5 = [1 + 4*n for n in layers]  # 5x5カーネル
    
    ax.plot(layers, rf_3x3, 'b-o', label='3×3カーネル')
    ax.plot(layers, rf_5x5, 'r-s', label='5×5カーネル')
    ax.axhline(y=25, color='green', linestyle='--', label='必要なRF=25')
    
    ax.set_xlabel('層数')
    ax.set_ylabel('受容野サイズ')
    ax.set_title('受容野の成長\n25ピクセルに達するまで', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 解決策
    ax = axes[2]
    solutions = [
        '1. 深い層を使う',
        '2. Dilated Convolution',
        '3. Poolingで解像度を下げる',
        '4. Self-Attention (ViT)',
    ]
    
    for i, sol in enumerate(solutions):
        ax.text(0.1, 0.8 - i*0.2, sol, fontsize=12, transform=ax.transAxes)
    
    ax.set_title('解決策', fontsize=12)
    ax.axis('off')
    
    plt.suptitle('長距離依存関係の問題', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

demonstrate_long_range_problem()

In [ ]:
def demonstrate_dilated_convolution():
    """Dilated Convolutionによる受容野拡大"""
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # 通常の畳み込み
    ax = axes[0]
    for i in range(7):
        for j in range(7):
            color = 'lightblue' if 2 <= i <= 4 and 2 <= j <= 4 else 'white'
            rect = plt.Rectangle((j, 6-i), 0.9, 0.9, facecolor=color, edgecolor='gray')
            ax.add_patch(rect)
    ax.set_xlim(-0.5, 7.5)
    ax.set_ylim(-0.5, 7.5)
    ax.set_aspect('equal')
    ax.set_title('通常の3×3畳み込み\nRF = 3', fontsize=12)
    ax.axis('off')
    
    # Dilation=2
    ax = axes[1]
    for i in range(7):
        for j in range(7):
            is_kernel = (i in [1, 3, 5]) and (j in [1, 3, 5])
            color = 'lightblue' if is_kernel else 'white'
            rect = plt.Rectangle((j, 6-i), 0.9, 0.9, facecolor=color, edgecolor='gray')
            ax.add_patch(rect)
    ax.set_xlim(-0.5, 7.5)
    ax.set_ylim(-0.5, 7.5)
    ax.set_aspect('equal')
    ax.set_title('Dilated Conv (d=2)\nRF = 5', fontsize=12)
    ax.axis('off')
    
    # Dilation=3
    ax = axes[2]
    for i in range(7):
        for j in range(7):
            is_kernel = (i in [0, 3, 6]) and (j in [0, 3, 6])
            color = 'lightblue' if is_kernel else 'white'
            rect = plt.Rectangle((j, 6-i), 0.9, 0.9, facecolor=color, edgecolor='gray')
            ax.add_patch(rect)
    ax.set_xlim(-0.5, 7.5)
    ax.set_ylim(-0.5, 7.5)
    ax.set_aspect('equal')
    ax.set_title('Dilated Conv (d=3)\nRF = 7', fontsize=12)
    ax.axis('off')
    
    plt.suptitle('Dilated Convolution: パラメータを増やさずに受容野を拡大', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

demonstrate_dilated_convolution()

<a id="section4"></a>
## 4. テクスチャバイアス

CNNは形状よりもテクスチャを重視する傾向があります（Geirhos et al., 2019）。

In [ ]:
def demonstrate_texture_bias():
    """テクスチャバイアスのデモ"""
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    np.random.seed(42)
    
    # 猫のシルエット（簡略）
    cat_shape = np.zeros((64, 64))
    cat_shape[20:50, 20:45] = 1  # 体
    cat_shape[10:25, 25:40] = 1  # 頭
    cat_shape[5:12, 22:28] = 1   # 左耳
    cat_shape[5:12, 35:41] = 1   # 右耳
    
    # 象のシルエット（簡略）
    elephant_shape = np.zeros((64, 64))
    elephant_shape[20:50, 15:55] = 1  # 体
    elephant_shape[15:35, 45:55] = 1  # 頭
    elephant_shape[35:55, 8:15] = 1   # 前脚
    elephant_shape[35:55, 48:55] = 1  # 後脚
    elephant_shape[20:30, 52:62] = 1  # 鼻
    
    # テクスチャ
    cat_texture = np.random.rand(64, 64) * 0.5 + 0.3  # 猫風（縞模様風）
    elephant_texture = np.random.rand(64, 64) * 0.3 + 0.5  # 象風（灰色）
    
    # 通常の画像
    axes[0, 0].imshow(cat_shape * cat_texture, cmap='gray', vmin=0, vmax=1)
    axes[0, 0].set_title('猫（形状）+ 猫テクスチャ\n→ 猫として認識', fontsize=11)
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(elephant_shape * elephant_texture, cmap='gray', vmin=0, vmax=1)
    axes[0, 1].set_title('象（形状）+ 象テクスチャ\n→ 象として認識', fontsize=11)
    axes[0, 1].axis('off')
    
    # テクスチャと形状を交換
    axes[1, 0].imshow(cat_shape * elephant_texture, cmap='gray', vmin=0, vmax=1)
    axes[1, 0].set_title('猫（形状）+ 象テクスチャ\n人間: 猫 / CNN: 象?', fontsize=11)
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(elephant_shape * cat_texture, cmap='gray', vmin=0, vmax=1)
    axes[1, 1].set_title('象（形状）+ 猫テクスチャ\n人間: 象 / CNN: 猫?', fontsize=11)
    axes[1, 1].axis('off')
    
    # 説明
    axes[0, 2].text(0.5, 0.5, 'テクスチャバイアス\n\nCNNは局所的な\nテクスチャパターンを\n重視しがち', 
                   fontsize=12, ha='center', va='center', transform=axes[0, 2].transAxes,
                   bbox=dict(boxstyle='round', facecolor='lightyellow'))
    axes[0, 2].axis('off')
    
    axes[1, 2].text(0.5, 0.5, '人間は形状を重視\nCNNはテクスチャを重視\n\n解決策:\n・Shape-biased訓練\n・データ拡張', 
                   fontsize=11, ha='center', va='center', transform=axes[1, 2].transAxes,
                   bbox=dict(boxstyle='round', facecolor='lightgreen'))
    axes[1, 2].axis('off')
    
    plt.suptitle('テクスチャバイアス：CNNと人間の認識の違い', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

demonstrate_texture_bias()

<a id="summary"></a>
## 5. まとめ

### CNNの限界

| 問題 | 原因 | 解決策 |
|------|------|--------|
| 回転に弱い | 平行移動のみ等変 | データ拡張、G-CNN |
| 長距離依存 | 受容野が局所的 | 深い層、Dilated Conv、Attention |
| テクスチャバイアス | 局所パターン重視 | Shape-biased訓練 |

### 重要なポイント

- **帰納バイアスは両刃の剣**: 適切なら効率的、不適切なら限界に
- **問題に応じたアーキテクチャ選択**が重要
- **Vision Transformer**など新しいアーキテクチャが登場

### 次のノートブック

次のノートブックでは、**CNNを超えて**ーVision TransformerやMLP-Mixerなど新しいアーキテクチャについて学びます。